In [1]:
!pip install tensorflow==2.4.0-rc0

     |████████████████████████████████| 394.7 MB 7.5 kB/s 
     |████████████████████████████████| 2.9 MB 50.3 MB/s 
     |████████████████████████████████| 3.8 MB 39.8 MB/s 
     |████████████████████████████████| 1.3 MB 27.9 MB/s 
     |████████████████████████████████| 459 kB 50.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found 

In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 4.1 MB/s 
     |████████████████████████████████| 486 kB 48.1 MB/s 
     |████████████████████████████████| 13.4 MB 209 kB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=4a02360c6d672dcc8446a5a838b39f1b97be1b98ea075d8a50cd8ea6e6b7d3e3
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.3 MB/s 


In [3]:
from transformers import BertJapaneseTokenizer, BertModel, T5Tokenizer

In [4]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

In [5]:
import os
import re
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics, losses
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import preprocessing as pp

In [6]:
ALL_SEED = 2021
def all_seed_fixed(seed=41):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
all_seed_fixed(ALL_SEED)

In [7]:
tf.__version__, keras.__version__

('2.4.0-rc0', '2.4.0')

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cd drive/MyDrive/

/content/drive/MyDrive


In [10]:
ls -al | grep dataset

-rw------- 1 root root 78862754 Aug 30 02:05 datasets.bin
-rw------- 1 root root  3976703 Aug 31 00:48 dev_datasets.bin


In [11]:
with open("./datasets.bin", "rb") as f:
  datasets = pickle.load(f)

In [12]:
len(datasets)

24218

In [13]:
df = pd.DataFrame(datasets, columns=["content", "title", "category", "tweet_share"])
df.dropna(inplace=True)
df.head()

,content,title,category,tweet_share
0,お笑いトリオ・安田大サーカスのクロちゃんの所属事務所・松竹芸能は19日、クロちゃんが新型コロ...,安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられます」,芸能,1025
1,元ユーチューバー、「へずまりゅう」こと原田将大被告――「…」先月、黒髪で上下黒のスーツを着て...,"へずまりゅうに懲役求刑“反省みられない""",国内,6
2,スピードを上げ…歩道に突進ドライブレコーダーが捉えたのは、コントロールを失った車がスピードを...,【独自】Uターン失敗でパニックに?歩道や花壇に次々と突っ込む…ドラレコが捉えた謎の暴走事故,国内,3
3,ミュージシャンの小山田圭吾が、学生時代に障がい者へのいじめを行っていたと告白した過去記事をめ...,小山田圭吾のいじめ告白記事『QuickJapan』出版元も謝罪「差別を助長する不適切なもの」,芸能,358
4,韓国の文在寅大統領の日本訪問と首脳会談について19日、韓国側は、不適切な発言を行ったソウルの...,韓国日本大使館幹部処分を首脳会談条件に,海外,28


In [14]:
df = df.loc[df.category != "None"]
print(df.shape)
df.head()

(24218, 4)


,content,title,category,tweet_share
0,お笑いトリオ・安田大サーカスのクロちゃんの所属事務所・松竹芸能は19日、クロちゃんが新型コロ...,安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられます」,芸能,1025
1,元ユーチューバー、「へずまりゅう」こと原田将大被告――「…」先月、黒髪で上下黒のスーツを着て...,"へずまりゅうに懲役求刑“反省みられない""",国内,6
2,スピードを上げ…歩道に突進ドライブレコーダーが捉えたのは、コントロールを失った車がスピードを...,【独自】Uターン失敗でパニックに?歩道や花壇に次々と突っ込む…ドラレコが捉えた謎の暴走事故,国内,3
3,ミュージシャンの小山田圭吾が、学生時代に障がい者へのいじめを行っていたと告白した過去記事をめ...,小山田圭吾のいじめ告白記事『QuickJapan』出版元も謝罪「差別を助長する不適切なもの」,芸能,358
4,韓国の文在寅大統領の日本訪問と首脳会談について19日、韓国側は、不適切な発言を行ったソウルの...,韓国日本大使館幹部処分を首脳会談条件に,海外,28


In [15]:
df.category.value_counts()

芸能              7150
IT 経済           3255
国内              3089
海外              2906
グルメ             2127
スポーツ            2029
ライフ総合           1169
ライフスタイル          938
恋愛               810
ファッション・ビューティ     467
車                278
Name: category, dtype: int64

In [16]:
# df = df.loc[df.category.isin(["IT 経済", "国内", "ライフ総合", "恋愛"])]
# df.shape

In [17]:
le = LabelEncoder()
df["category_idx"] = le.fit_transform(df.category.values)
df.head()

,content,title,category,tweet_share,category_idx
0,お笑いトリオ・安田大サーカスのクロちゃんの所属事務所・松竹芸能は19日、クロちゃんが新型コロ...,安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられます」,芸能,1025,9
1,元ユーチューバー、「へずまりゅう」こと原田将大被告――「…」先月、黒髪で上下黒のスーツを着て...,"へずまりゅうに懲役求刑“反省みられない""",国内,6,6
2,スピードを上げ…歩道に突進ドライブレコーダーが捉えたのは、コントロールを失った車がスピードを...,【独自】Uターン失敗でパニックに?歩道や花壇に次々と突っ込む…ドラレコが捉えた謎の暴走事故,国内,3,6
3,ミュージシャンの小山田圭吾が、学生時代に障がい者へのいじめを行っていたと告白した過去記事をめ...,小山田圭吾のいじめ告白記事『QuickJapan』出版元も謝罪「差別を助長する不適切なもの」,芸能,358,9
4,韓国の文在寅大統領の日本訪問と首脳会談について19日、韓国側は、不適切な発言を行ったソウルの...,韓国日本大使館幹部処分を首脳会談条件に,海外,28,8


In [18]:
df.drop_duplicates(inplace=True)
df.shape

(23913, 5)

In [19]:
" ".join(tokenizer.tokenize(df.title.iloc[0]))

'安田 大 サーカス ・ クロ ちゃん 、 新型 コロナ 感染 団長 安田 と は 「 別 ルート で の 感染 と 考え られ ます 」'

In [20]:
df["content_"] = df.content.apply(lambda x: " ".join(tokenizer.tokenize(x)))
df["title_"] = df.title.apply(lambda x: "[BOS] " + " ".join(tokenizer.tokenize(x)) + " [EOS]")
df.head()

,content,title,category,tweet_share,category_idx,content_,title_
0,お笑いトリオ・安田大サーカスのクロちゃんの所属事務所・松竹芸能は19日、クロちゃんが新型コロ...,安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられます」,芸能,1025,9,お笑い トリオ ・ 安田 大 サーカス の クロ ちゃん の 所属 事務所 ・ 松竹 芸能 ...,[BOS] 安田 大 サーカス ・ クロ ちゃん 、 新型 コロナ 感染 団長 安田 と は...
1,元ユーチューバー、「へずまりゅう」こと原田将大被告――「…」先月、黒髪で上下黒のスーツを着て...,"へずまりゅうに懲役求刑“反省みられない""",国内,6,6,元 ユー ##チュー ##バー 、 「 へ ず ま りゅう 」 こと 原田 将 大 被告 ―...,"[BOS] へ ##ず ま りゅう に 懲役 求 ##刑 “ 反省 み られ ない "" [EOS]"
2,スピードを上げ…歩道に突進ドライブレコーダーが捉えたのは、コントロールを失った車がスピードを...,【独自】Uターン失敗でパニックに?歩道や花壇に次々と突っ込む…ドラレコが捉えた謎の暴走事故,国内,3,6,スピード を 上げ ... 歩道 に 突 ##進 ドライブ レコーダー が 捉え た の は...,[BOS] 【 独自 】 U ターン 失敗 で パニック に ? 歩道 や 花 ##壇 に ...
3,ミュージシャンの小山田圭吾が、学生時代に障がい者へのいじめを行っていたと告白した過去記事をめ...,小山田圭吾のいじめ告白記事『QuickJapan』出版元も謝罪「差別を助長する不適切なもの」,芸能,358,9,ミュージシャン の 小山 ##田 圭 ##吾 が 、 学生 時代 に 障 が い 者 へ の...,[BOS] 小山 ##田 圭 ##吾 の いじめ 告白 記事 『 Qu ##ick ##J ...
4,韓国の文在寅大統領の日本訪問と首脳会談について19日、韓国側は、不適切な発言を行ったソウルの...,韓国日本大使館幹部処分を首脳会談条件に,海外,28,8,韓国 の 文 在 寅 大統領 の 日本 訪問 と 首脳 会談 について 19 日 、 韓国 ...,[BOS] 韓国 日本 大使館 幹部 処分 を 首脳 会談 条件 に [EOS]


In [21]:
keras_tokenizer = pp.text.Tokenizer(filters="", lower=False)
keras_tokenizer.fit_on_texts(df.content_.to_list() + df.title_.to_list())

In [22]:
print(len(keras_tokenizer.word_index))
keras_tokenizer.word_index[""] = 0
keras_tokenizer.index_word[0] = ""
print(len(keras_tokenizer.word_index))

29390
29391


In [23]:
with open("./keras_tokenizer.bin", 'wb') as f:
    pickle.dump(keras_tokenizer , f)

## Train, Val Split

In [24]:
train_df, val_df = train_test_split(df, test_size=0.05, random_state=2021)
train_df.shape, val_df.shape

((22717, 7), (1196, 7))

In [25]:
train_content = keras_tokenizer.texts_to_sequences(train_df.content_.to_list())
val_content = keras_tokenizer.texts_to_sequences(val_df.content_.to_list())
train_title = keras_tokenizer.texts_to_sequences(train_df.title_.to_list())
val_title = keras_tokenizer.texts_to_sequences(val_df.title_.to_list())

In [26]:
" ".join([keras_tokenizer.index_word[idx] for idx in train_content[0]]), "->" , " ".join([keras_tokenizer.index_word[idx] for idx in train_title[0]])

('今や ドラッグ ##ストア など で も 気 ##軽 に 買 ##える 「 レ ##チ ##ノール 」 配合 の コス ##メ 。 年齢 サイン に アプローチ する 優れ た 成分 という の は なんと ##なく わかる けれ ##ど 、 使う とき の 注意 点 など 知ら ない こと も 多い の で は ? そこ で 、 肌 を 知り 尽くし た エキス ##パート 、 美容 皮膚 科 タカ ##ミ ##クリ ##ニック 副 院長 の 山 ##屋 医師 に 「 レ ##チ ##ノール 」 について の あれ ##こ ##れ を 伺 ##い まし た ! レ ##チ ##ノール と は ? 山 ##屋 医師 に よる と 、 レ ##チ ##ノール は ビタミン A の 一種 と の こと 。 そして 化粧 品 や 医薬 部 ##外 品 、 医療 品 に 配合 さ れ て いる ビタミン A に は 、 大きく 分け て 2 つ の 種類 が ある よう 。 レ ##チ ##ノール ##トレ ##チ ##ノイ ##ン ##レチ ##ノール は 、 ビタミン A の 中 でも 肌 へ の 負担 が 少ない ため 、 日常 でも 使い やすく 、 化粧 品 や 医薬 部 ##外 品 に 使用 さ れ て いる ん だ と か 。 また トレ ##チ ##ノイ ##ン も ビタミン A の 一種 だ けど 、 レ ##チ ##ノール と は 別物 で ある と 山 ##屋 医師 は 解説 。 「 トレ ##チ ##ノイ ##ン は レ ##チ ##ノール の 50 ~ 100 倍 の 生理 活性 を 有し て い ます が 、 副作用 も 強い ため 、 医師 の 診察 により 適切 な 濃度 の もの を 処方 し て もらう 必要 が あり ます 」 レ ##チ ##ノール に 期待 できる 効果 それで ##は 、 レ ##チ ##ノール 製品 を 使用 する こと で どの よう な 効果 が 得 られる の でしょ う か ? 山 ##屋 医師 に 聞い て み まし た ! 角 ##質 の 代謝 を 促進 し て 、 ターン オーバー を 正常 化 できる 。 真 ##皮 の 働き を 活性 化 し て コラ ##ー ##ゲン 生成 を 促進 し 

In [28]:
ENC_SEQ_LEN = 1400
DEC_SEQ_LEN = 50

# 入力
train_content = pp.sequence.pad_sequences(train_content, maxlen=ENC_SEQ_LEN, padding="post", truncating="post")
val_content = pp.sequence.pad_sequences(val_content, maxlen=ENC_SEQ_LEN, padding="post", truncating="post")
# 出力
train_title = pp.sequence.pad_sequences(train_title, maxlen=DEC_SEQ_LEN+1, padding="post", truncating="post")
val_title = pp.sequence.pad_sequences(val_title, maxlen=DEC_SEQ_LEN+1, padding="post", truncating="post")

In [29]:
train_title_enc, val_title_enc = train_title[:,:-1], val_title[:,:-1]
train_title_dec, val_title_dec = train_title[:,1:], val_title[:,1:]

In [30]:
# 文書分類ラベル
train_category = train_df.category_idx.values
val_category = val_df.category_idx.values
# # 文書人気予測ラベル
# train_tweet = train_df.tweet_share.values
# val_tweet = val_df.tweet_share.values

train_category.shape, val_category.shape

((22717,), (1196,))

In [31]:
print(train_content.shape, train_title_enc.shape, train_title_dec.shape, train_category.shape)
print()
print(val_content.shape, val_title_enc.shape, val_title_dec.shape, val_category.shape)

(22717, 1400) (22717, 50) (22717, 50) (22717,)

(1196, 1400) (1196, 50) (1196, 50) (1196,)


In [32]:
# Constant Value
VOCAB_SIZE = len(keras_tokenizer.word_index)
EMB_DIM = 192
UNITS_DIM = 128
# inputs
encoder_inputs = keras.Input(shape=(ENC_SEQ_LEN,), dtype="int32", name="encoder_inputs")
decoder_inputs = keras.Input(shape=(DEC_SEQ_LEN,), dtype="int32", name="decoder_inputs")
enc_dec_emb = layers.Embedding(VOCAB_SIZE, EMB_DIM)
# feed forward
output, state = layers.GRU(UNITS_DIM, return_sequences=True, return_state=True)


# feed_foward
enc_output, enc_output_cls = tf_encoder(pos_emb(encoder_inputs), enc_padding_mask)
dec_output = tf_decoder(pos_emb(decoder_inputs), enc_output, combined_mask, dec_padding_mask)
# global model
trainer = keras.Model([encoder_inputs, decoder_inputs], [dec_output, enc_output_cls], name="s2s")


NameError: ignored

In [ ]:
dddddd

In [ ]:
tf_encoder.summary()

Model: "encoder_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_layer_13 (EncoderLay multiple                  322200    
_________________________________________________________________
encoder_layer_14 (EncoderLay multiple                  322200    
_________________________________________________________________
global_average_pooling1d_6 ( multiple                  0         
_________________________________________________________________
sequential_38 (Sequential)   (None, 11)                84811     
Total params: 729,211
Trainable params: 729,211
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf_decoder.summary()

Model: "decoder_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_layer_12 (DecoderLay multiple                  483400    
_________________________________________________________________
decoder_layer_13 (DecoderLay multiple                  483400    
_________________________________________________________________
sequential_41 (Sequential)   (None, 50, 29007)         5830407   
Total params: 6,797,207
Trainable params: 6,797,207
Non-trainable params: 0
_________________________________________________________________


In [ ]:
trainer.summary()

Model: "s2s"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     [(None, 1400)]       0                                            
__________________________________________________________________________________________________
positional_encoding_7 (Position multiple             5801400     encoder_inputs[0][0]             
                                                                 decoder_inputs[0][0]             
__________________________________________________________________________________________________
lambda_7 (Lambda)               ((None, 1, 1, 1400), 0           encoder_inputs[0][0]           

In [ ]:
def word_loss(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  # reductionはlossをsumせずに個々に保持すること
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
enumerate

enumerate

In [ ]:
class GenerateTitleCallback(keras.callbacks.Callback):
  def __init__(self, x_val, y_val):
    self.x_val = x_val
    self.y_val = y_val

  def on_epoch_end(self, epoch, logs=None):
    # バリデーションデータの中からランダムで見出しを生成する
    sampling_idx = np.random.randint(0, self.x_val.shape[0])
    sample_enc_inp = self.x_val[sampling_idx-1:sampling_idx, :]
    decoder_sentence = "[BOS]"
    for i in range(50):
      decoder_sentence_ = keras_tokenizer.texts_to_sequences([decoder_sentence])
      decoder_sentence_ = pp.sequence.pad_sequences(decoder_sentence_, maxlen=DEC_SEQ_LEN, padding="post", truncating="post")
      pred, category_pred = trainer((sample_enc_inp, decoder_sentence_))
      sampled_tkn_idx = np.argmax(pred[0,i,:])
      sampled_tkn = keras_tokenizer.index_word[sampled_tkn_idx]
      decoder_sentence =  decoder_sentence + " " + sampled_tkn
      if sampled_tkn == "[EOS]":
        break
    category_label = le.classes_[np.argmax(category_pred[0])]
    pred_title = "".join(re.sub("#", "", decoder_sentence).split(" ")[1:-1])
    raw_content = re.sub("#", "", "".join([keras_tokenizer.index_word[idx] for idx in sample_enc_inp[0]]))
    print(raw_content)
    print("↓")
    print(category_label)
    print("↓")
    print(pred_title)

call_backs = GenerateTitleCallback(val_content, val_title_enc)

In [ ]:
trainer.compile(
    "adam",
    loss=[word_loss, losses.SparseCategoricalCrossentropy(from_logits=True)],
    metrics=[[metrics.SparseCategoricalAccuracy(name='word_acc')], [metrics.SparseCategoricalAccuracy(name="category_acc")]],
    loss_weights=[9, 1]
    )

In [ ]:
VOCAB_SIZE

29007

In [ ]:
trainer.fit(
    x=(train_content, train_title_enc), y=(train_title_dec, train_category), 
    validation_data=((val_content, val_title_enc), (val_title_dec, val_category)),
    batch_size=64, epochs=20,
    callbacks=[call_backs]
    )

Epoch 1/20
278/278 [==============================] - 188s 676ms/step - loss: 3.9717 - decoder_6_loss: 0.4396 - encoder_7_loss: 0.0152 - decoder_6_word_acc: 0.3683 - encoder_7_category_acc: 0.9953 - val_loss: 36.9453 - val_decoder_6_loss: 3.9267 - val_encoder_7_loss: 1.6048 - val_decoder_6_word_acc: 0.1383 - val_encoder_7_category_acc: 0.8191
StrayKidsがニューアルバム「NOEASY」の収録曲「好きでごめん」の予告映像で感性溢れる魅力を披露した。本日、公式SNSにを通じて「StrayKids<NOEASY>UNVEIL:TRACK6“好きでごめん"」が掲載された。カムバックを控えて、ニューアルバムの収録曲の一部を紹介するシグネチャーティージングコンテンツ「UNVEIL:TRACK」の映像第6弾であり、メンバーたちの声色が強調されて関心を集めた。スタンドマイクの前に立ったメンバーたちは「好きなのが罪になるならとても好きでごめん/表に出さないように努力しても全部見えてごめん」のような現実感のある初々しい心を描いた歌詞を淡々と歌った。ライブの雰囲気を活かし、感性溢れるラップとボーカルの魅力を高めた。世界中のファンが彼らの多彩な魅力に熱い反応を見せた。2ndフルアルバム「NOEASY」は、壮大なメロディーに伝統国楽の華やかで多彩な音を加えたタイトル曲「ソリクン」から叙情的な雰囲気の収録曲まで幅広いジャンルが盛り込まれ、聴く楽しさを高めている。グループのプロデュースユニット“3RACHA"のバンチャン、チャンビン、ハンを筆頭に、メンバー全員が新譜のクレジットに名を連ねた。他にもVERSACHOI、KrystaYoungs、HotSauce、ホン・ジサンなど、韓国国内外の有数のアーティストが意気投合した。今年6月にMnet「KINGDOM:LEGENDARYWAR」で最終優勝を果たしたStrayKidsは、優勝の恩恵として今月17〜

In [ ]:
# ひろゆきのサンプルいいかも
# メインヒロイン

In [ ]:
trainer.save_weights("./trainer_weight_multi.h5")

In [ ]:
trainer.load_weights("./trainer_weight_multi.h5")

In [ ]:
le.classes_

array(['IT 経済', 'グルメ', 'スポーツ', 'ファッション・ビューティ', 'ライフスタイル', 'ライフ総合', '国内',
       '恋愛', '海外', '芸能', '車'], dtype=object)

In [ ]:
trainer.layers

In [ ]:
def decode_title(input_sentence):
  decoder_sentence = "[BOS]"
  input_sentence = " ".join(tokenizer.tokenize(input_sentence))
  encoder_sentence_ = keras_tokenizer.texts_to_sequences([input_sentence])
  encoder_sentence_ = pp.sequence.pad_sequences(encoder_sentence_, maxlen=ENC_SEQ_LEN, padding="post", truncating="post")
  for i in range(50):
    decoder_sentence_ = keras_tokenizer.texts_to_sequences([decoder_sentence])
    decoder_sentence_ = pp.sequence.pad_sequences(decoder_sentence_, maxlen=DEC_SEQ_LEN, padding="post", truncating="post")
    pred, _ = trainer((encoder_sentence_, decoder_sentence_))
    sampled_tkn_idx = np.argmax(pred[0,i,:])
    sampled_tkn = keras_tokenizer.index_word[sampled_tkn_idx]
    decoder_sentence =  decoder_sentence + " " + sampled_tkn
    if sampled_tkn == "[EOS]":
      break
  return "".join(re.sub("#", "", decoder_sentence).split(" ")[1:-1])

In [ ]:
idx = np.random.randint(0, len(datasets))
print(datasets[idx][0])
print("↓")
print(decode_title(datasets[idx][0]))
print("↓")
print(datasets[idx][1])

[11日ロイター]-豪コモンウェルス銀行は11日、過去最高となる60億豪ドル規模の自社株買いを発表した。新型コロナウイルス流行からの景気回復で通期キャッシュ利益が約20%増えたことを受けた。オーストラリアの新型コロナ流行抑制とそれに伴う早期の景気回復が銀行を支援。また、このところの非中核部門の売却で「豪4大銀行」はキャッシュが豊富となっており、各行は向こう2年間で過去最高となる260億豪ドルの株主還元を見込んでいる。CBAは国内経済の耐性および自社株買い後の潜在的なストレスイベントを吸収できる自行の能力を考慮したと説明。今回の自社株買いでは発行済み株式資本が3.5%超減少する。アナリストは国内最大手行であるCBAの自社株買いについて、50億豪ドル規模になると予想していた。競合のオーストラリア・アンド・ニュージーランド・バンキング・グループとナショナル・オーストラリア銀行は先月、合計で40億豪ドル相当の自社株買い計画を発表した。CBAのマット・コミン最高経営責任者は、2021会計年度における豪景気回復の力強さを認識しつつ、新型コロナ流行が引き続き経済に影響を及ぼすほか、金利低下が自行の将来の利益にとって圧力になると指摘した。6月30日までの会計年度では、CBAの税引き後キャッシュ純利益は19.8%増の86億5000万豪ドル。アナリスト5人によるコンセンサス予想を上回った。最終配当は1株当たり2.00豪ドルとした。規制当局が配当を制限していた昨年は98豪セントだった。配当性向はCBAのキャッシュ利益の71%に相当。同行は引き続き通期配当性向目標をキャッシュ利益の70─80%にするとした。融資減損費用は5億5400万豪ドルとなり、昨年の25億2000万豪ドルから減少した。
↓
豪CBA、44億ドル規模の自社株買い発表景気回復で利益が拡大
↓
豪CBA、44億ドル規模の自社株買い発表景気回復で利益が拡大


In [ ]:
news_sample = '''比嘉愛未（35）主演の連ドラ「推しの王子様」（フジテレビ系木曜夜10時）。乙女ゲームを手がけるベンチャー企業の女社長・泉美が、自身の理想通りにつくった推しキャラのケント様にそっくりな航（渡辺圭祐＝27）に出会うことで始まる物語だが、当初主演予定だった深田恭子（38）が適応障害で休業することを公表し、降板。急きょ代役で泉美を演じることになったのが、比嘉だ。

比嘉愛未「2行のセリフ」が言えなかった駆け出し時代

「最近の代役といえば、NHK大河『麒麟がくる』でしょう。沢尻エリカの代役で、川口春奈が帰蝶を演じましたが、ふたを開けてみると、〈川口でよかった〉の声がほとんどだった。『推しの王子様』の比嘉も、初回を見る限り、そういう評価になりそうな予感がします。仕事ができる女社長役って、深田よりむしろ比嘉の方がハマる気がしますもんね」（ドラマ制作会社スタッフ）'''

In [ ]:
news_sample = re.sub(r"\s", "", news_sample)
news_sample = re.sub("\u3000", "", news_sample)
news_sample

'比嘉愛未（35）主演の連ドラ「推しの王子様」（フジテレビ系木曜夜10時）。乙女ゲームを手がけるベンチャー企業の女社長・泉美が、自身の理想通りにつくった推しキャラのケント様にそっくりな航（渡辺圭祐＝27）に出会うことで始まる物語だが、当初主演予定だった深田恭子（38）が適応障害で休業することを公表し、降板。急きょ代役で泉美を演じることになったのが、比嘉だ。比嘉愛未「2行のセリフ」が言えなかった駆け出し時代「最近の代役といえば、NHK大河『麒麟がくる』でしょう。沢尻エリカの代役で、川口春奈が帰蝶を演じましたが、ふたを開けてみると、〈川口でよかった〉の声がほとんどだった。『推しの王子様』の比嘉も、初回を見る限り、そういう評価になりそうな予感がします。仕事ができる女社長役って、深田よりむしろ比嘉の方がハマる気がしますもんね」（ドラマ制作会社スタッフ）'

In [ ]:
decode_title(news_sample)

'『ウマ娘』次回「レオ杯」8月下旬開催!阪神、芝...気になるレース環境は?'

In [ ]:
re.sub().split(" ")[1:-1]

In [ ]:
re.sub?